In [1]:
import base64
import bz2
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

model_path = "models/latest.pth"

weights = torch.load(model_path)

PARAM = base64.b64encode(bz2.compress(pickle.dumps(weights)))
state_dict = pickle.loads(bz2.decompress(base64.b64decode(PARAM)))

In [2]:
%%writefile submission.py

# This is a lightweight ML agent trained by self-play.
# After sharing this notebook,
# we will add Hungry Geese environment in our HandyRL library.
# https://github.com/DeNA/HandyRL
# We hope you enjoy reinforcement learning!


import base64
import bz2
import math
import pickle
from collections import defaultdict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# Neural Network for Hungry Geese


class Dense(nn.Module):
    def __init__(self, units0, units1, bnunits=0, bias=True):
        super().__init__()
        if bnunits > 0:
            bias = False
        self.dense = nn.Linear(units0, units1, bias=bias)
        self.bnunits = bnunits
        self.bn = nn.BatchNorm1d(bnunits) if bnunits > 0 else None

    def forward(self, x):
        h = self.dense(x)
        if self.bn is not None:
            size = h.size()
            h = h.view(-1, self.bnunits)
            h = self.bn(h)
            h = h.view(*size)
        return h


class TorusConv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.edge_size = (kernel_size[0] // 2, kernel_size[1] // 2)
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size)
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = torch.cat([x[:, :, :, -self.edge_size[1] :], x, x[:, :, :, : self.edge_size[1]]], dim=3)
        h = torch.cat([h[:, :, -self.edge_size[0] :], h, h[:, :, : self.edge_size[0]]], dim=2)
        h = self.conv(h)
        h = self.bn(h) if self.bn is not None else h
        return h


class Conv2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, bn):
        super().__init__()
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size=kernel_size)
        self.bn = nn.BatchNorm2d(output_dim) if bn else None

    def forward(self, x):
        h = self.conv(x)
        h = self.bn(h) if self.bn is not None else h
        return h


class ChannelSELayer(nn.Module):
    def __init__(self, channel, reduction=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


# https://github.com/Kaixhin/Rainbow/blob/master/model.py
# Factorised NoisyLinear layer with bias
class NoisyLinear(nn.Module):
    def __init__(self, in_features, out_features, std_init=0.5):
        super(NoisyLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.std_init = std_init
        self.weight_mu = nn.Parameter(torch.empty(out_features, in_features))
        self.weight_sigma = nn.Parameter(torch.empty(out_features, in_features))
        self.register_buffer("weight_epsilon", torch.empty(out_features, in_features))
        self.bias_mu = nn.Parameter(torch.empty(out_features))
        self.bias_sigma = nn.Parameter(torch.empty(out_features))
        self.register_buffer("bias_epsilon", torch.empty(out_features))
        self.reset_parameters()
        self.reset_noise()

    def reset_parameters(self):
        mu_range = 1 / math.sqrt(self.in_features)
        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(self.std_init / math.sqrt(self.in_features))
        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(self.std_init / math.sqrt(self.out_features))

    def _scale_noise(self, size):
        x = torch.randn(size, device=self.weight_mu.device)
        return x.sign().mul_(x.abs().sqrt_())

    def reset_noise(self):
        epsilon_in = self._scale_noise(self.in_features)
        epsilon_out = self._scale_noise(self.out_features)
        self.weight_epsilon.copy_(epsilon_out.ger(epsilon_in))
        self.bias_epsilon.copy_(epsilon_out)

    def forward(self, input):
        if self.training:
            return F.linear(
                input,
                self.weight_mu + self.weight_sigma * self.weight_epsilon,
                self.bias_mu + self.bias_sigma * self.bias_epsilon,
            )
        else:
            return F.linear(input, self.weight_mu, self.bias_mu)


class GeeseNet(nn.Module):
    def __init__(self):
        super().__init__()
        layers, filters = 6, 40

        self.conv0 = TorusConv2d(17, filters, (3, 3), True)
        self.cnn_blocks = nn.ModuleList([TorusConv2d(filters, filters, (3, 3), True) for _ in range(layers)])
        self.cse_blocks = nn.ModuleList([ChannelSELayer(filters, reduction=4) for _ in range(layers)])

        self.conv_p = TorusConv2d(filters, filters, (3, 3), True)
        self.conv_v = TorusConv2d(filters, filters, (3, 3), True)

        self.head_p = NoisyLinear(filters, 4)
        self.head_v1 = NoisyLinear(77, 16)
        self.head_v2 = NoisyLinear(16, 1)

    def forward(self, x, _=None):
        h = F.relu_(self.conv0(x))
        for cnn, cse in zip(self.cnn_blocks, self.cse_blocks):
            h = cnn(h)
            h = F.relu_(h + cse(h))

        p = F.relu_(self.conv_p(h))
        head = x[:, :1]
        p_head = (p * head).view(h.size(0), h.size(1), -1).sum(-1)
        p = self.head_p(p_head)

        v = F.relu_(self.conv_v(h))
        v_gap = v.view(h.size(0), h.size(1), -1).mean(1)
        v = F.relu_(self.head_v1(v_gap))
        v = torch.tanh(self.head_v2(v))

        return {"policy": p, "value": v}

    def reset_noise(self):
        for name, module in self.named_children():
            if "head" in name:
                module.reset_noise()


# Input for Neural Network


NUM_ROW = 7
NUM_COL = 11
CENTER_ROW = NUM_ROW // 2
CENTER_COL = NUM_COL // 2


def to_offset(x):
    row = CENTER_ROW - x // NUM_COL
    col = CENTER_COL - x % NUM_COL
    return row, col


def to_row(offset, x):
    return (x // NUM_COL + offset) % NUM_ROW


def to_col(offset, x):
    return (x + offset) % NUM_COL


def make_input(obses):
    b = np.zeros((17, 7 * 11), dtype=np.float32)
    obs = obses[-1]

    for p, pos_list in enumerate(obs["geese"]):
        # head position
        for pos in pos_list[:1]:
            b[0 + (p - obs["index"]) % 4, pos] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[4 + (p - obs["index"]) % 4, pos] = 1
        # whole position
        for pos in pos_list:
            b[8 + (p - obs["index"]) % 4, pos] = 1

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, pos_list in enumerate(obs_prev["geese"]):
            for pos in pos_list[:1]:
                b[12 + (p - obs["index"]) % 4, pos] = 1

    # food
    for pos in obs["food"]:
        b[16, pos] = 1

    return b.reshape(-1, 7, 11)


def make_input_centering_head(obses):
    b = np.zeros((17, 7, 11), dtype=np.float32)
    obs = obses[-1]

    player_goose_head = obs["geese"][obs["index"]][0]
    o_row, o_col = to_offset(player_goose_head)

    for p, geese in enumerate(obs["geese"]):
        # whole position
        for pos in geese:
            b[0 + (p - obs["index"]) % 4, to_row(o_row, pos), to_col(o_col, pos)] = 1
        # tip position
        for pos in geese[-1:]:
            b[4 + (p - obs["index"]) % 4, to_row(o_row, pos), to_col(o_col, pos)] = 1
        # head position
        for pos in geese[:1]:
            b[8 + (p - obs["index"]) % 4, to_row(o_row, pos), to_col(o_col, pos)] = 1

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, geese in enumerate(obs_prev["geese"]):
            for pos in geese[:1]:
                b[12 + (p - obs["index"]) % 4, to_row(o_row, pos), to_col(o_col, pos)] = 1

    # food
    for pos in obs["food"]:
        b[16, to_row(o_row, pos), to_col(o_col, pos)] = 1

    return b


def make_input_centering_head_for_rule(obses):
    b = defaultdict(list)
    obs = obses[-1]

    player_goose_head = obs["geese"][obs["index"]][0]
    o_row, o_col = to_offset(player_goose_head)

    for p, geese in enumerate(obs["geese"]):
        # body position
        for pos in geese[1:-1]:
            if (p - obs["index"]) % 4 == 0:
                b["pb"].append((to_row(o_row, pos), to_col(o_col, pos)))
            else:
                b["ob"].append((to_row(o_row, pos), to_col(o_col, pos)))

        # tip position
        for pos in geese[-1:]:
            if (p - obs["index"]) % 4 == 0:
                b["pt"].append((to_row(o_row, pos), to_col(o_col, pos)))
            else:
                b["ot"].append((to_row(o_row, pos), to_col(o_col, pos)))

        # head position
        for pos in geese[:1]:
            if (p - obs["index"]) % 4 == 0:
                b["ph"].append((to_row(o_row, pos), to_col(o_col, pos)))
            else:
                b["oh"].append((to_row(o_row, pos), to_col(o_col, pos)))

    # previous head position
    if len(obses) > 1:
        obs_prev = obses[-2]
        for p, geese in enumerate(obs_prev["geese"]):
            for pos in geese[:1]:
                if (p - obs["index"]) % 4 == 0:
                    b["pp"].append((to_row(o_row, pos), to_col(o_col, pos)))

    # food
    for pos in obs["food"]:
        b["f"].append((to_row(o_row, pos), to_col(o_col, pos)))

    return b


def distance(a, b):
    x = b[0] - a[0]
    y = b[1] - a[1]
    return (x, y), abs(x) + abs(y)


def apply_rule(b, prob):
    """
    player head = (3, 5)
    ["NORTH", "SOUTH", "WEST", "EAST"]
    """
    neighbor = [(2, 5), (4, 5), (3, 4), (3, 6)]

    # 隣接している場所に行けないケース
    for i, n in enumerate(neighbor):
        if (n in b["pb"]) or (n in b["ob"]) or (n in b["pp"]):
            prob[i] = -np.inf

    # 次の移動で頭がぶつかる可能性のあるケース
    for i, h in enumerate(b["oh"]):
        (x, y), d = distance(b["ph"][0], h)
        if d == 2:
            if x < 0:
                prob[0] -= 10
            elif x > 0:
                prob[1] -= 10
            if y < 0:
                prob[2] -= 10
            elif y > 0:
                prob[3] -= 10

    return prob


# Load PyTorch Model


PARAM = b"xxxxxxxxxx"

state_dict = pickle.loads(bz2.decompress(base64.b64decode(PARAM)))
model = GeeseNet()
# model = GeeseNetA()
model.load_state_dict(state_dict)
model.eval()


# Main Function of Agent

obses = []


def agent(obs, _):
    obses.append(obs)

    x = make_input(obses)
    # x = make_input_centering_head(obses)
    y = make_input_centering_head_for_rule(obses)

    with torch.no_grad():
        xt = torch.from_numpy(x).unsqueeze(0)
        o = model(xt)
    p = o["policy"].squeeze(0).detach().numpy()

    p = apply_rule(y, p)

    print(p)

    actions = ["NORTH", "SOUTH", "WEST", "EAST"]
    return actions[np.argmax(p)]

Overwriting submission.py


In [3]:
# Read in the submission file
with open(
    "submission.py",
) as file:
    filedata = file.read()

# Replace the target string
filedata = filedata.replace("xxxxxxxxxx", PARAM.decode("utf-8"))

# Write the file out again
with open("submission.py", "w") as file:
    file.write(filedata)

In [4]:
from kaggle_environments import make

env = make("hungry_geese", debug=True)

env.reset()
env.run(["submission.py", "submission.py", "submission.py", "submission.py"])
env.render(mode="ipython", width=800, height=700)

Loading environment football failed: No module named 'gfootball'
[-0.64803153 -0.0695943   0.51315284 -0.45491347]
[ 0.24605624  0.25899968 -1.1375308  -0.18170054]
[-1.0003957e+04  1.6424626e+00  1.4911733e+00 -1.0005652e+04]
[ 2.2024882e+00 -1.0003889e+04 -1.0003497e+04  1.3600917e+00]
[1.8142788 1.5332037 0.0208315      -inf]
[       -inf  0.21752834 -1.8158584   3.625558  ]
[      -inf 0.886912   1.0369767  0.72019047]
[0.98370785       -inf 1.5572426  0.42878348]
[ 2.9780045e+00           -inf -1.0000932e+04  8.4870642e-01]
[ 4.0546851e+00  1.3322862e+00           -inf -1.0003319e+04]
[ 1.2703227   1.1360751  -0.16912082        -inf]
[2.5506537 0.9232602 1.5244358      -inf]
[ 3.3264084e+00           -inf -1.0003006e+04  1.5644162e+00]
[ 5.1710944e+00           -inf  5.7764349e+00 -1.0010226e+04]
[0.34960705       -inf 1.1712958  0.39171183]
[-0.8406853         -inf  2.058612    0.86129695]
[ 2.275442        -inf  1.2947867 -0.5919402]
[ 6.026534  -1.9361588  0.6884556       -inf]